# UKWA Stats Module

Copyright 2023 Michael George (AKA Logiqx).

This file is part of [wsw-results](https://github.com/Logiqx/wsw-results) and is distributed under the terms of the GNU General Public License.

wsw-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

wsw-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

### Summary

Adhoc script to determine the correctness of UKWA flags in the current year

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import glob

import csv
import math

import numpy as np

from datetime import datetime

In [2]:
PROJDIR = os.path.realpath(os.path.join(sys.path[0], '..'))

## Process Entrants

Process all names

In [3]:
def compare_25_50(year, csvFile):
    '''processSession processes an individual session for an event'''

    session = os.path.basename(os.path.dirname(csvFile))
    
    RUN_2_KTS = 'Run 2 (kts)'
    AVG_KTS = 'Avg (kts)'

    results = []
    runs = []
    averages = []
    with open(csvFile, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for result in csvReader:
            results.append(result)
            try:
                runs.append(float(result[RUN_2_KTS]))
            except ValueError:
                runs.append(0)
            averages.append(float(result[AVG_KTS]))
            
    runs = sorted(runs, reverse=True)
            
    percent25 = math.ceil(len(runs) / 4)
    percent50 = math.ceil(len(runs) / 2)
    
    if runs[percent25 - 1] >= 28 and runs[percent50 - 1] >= 26:
        #print(session, 'valid-both', len(results), runs[percent25 - 1], runs[percent50 - 1])
        pass

    if runs[percent50 - 1]:
        if runs[percent25 - 1] >= 28:
            #print(session, 'valid-25', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass
        elif runs[percent50 - 1] >= 26:
            print(session, 'valid-50', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass
        else:
            #print(session, 'invalid', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass

    if runs[percent25 - 1]:
        if runs[percent50 - 1] >= 26:
            #print(session, 'valid-50', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass
        elif runs[percent25 - 1] >= 28:
            print(session, 'valid-25', len(results), runs[percent25 - 1], runs[percent50 - 1])
        else:
            #print(session, 'invalid', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass

In [4]:
def processSession(year, csvFile):
    '''processSession processes an individual session for an event'''

    session = os.path.basename(os.path.dirname(csvFile))
    
    RUN_2_KTS = 'Run 2 (kts)'
    AVG_KTS = 'Avg (kts)'
    STATUS = 'Status'

    results = []
    with open(csvFile, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for result in csvReader:
            results.append(result)

    allRuns = []
    for result in results:
        try:
            allRuns.append(float(result[RUN_2_KTS]))
        except ValueError:
            allRuns.append(0)
            
    allRuns = sorted(allRuns, reverse=True)
            
    percent25 = math.ceil(len(allRuns) / 4)
    percent50 = math.ceil(len(allRuns) / 2)
    
    if allRuns[percent25 - 1] >= 28: # or allRuns[percent50 - 1] >= 26:
        oldRule = True
    else:
        oldRule = False

    silverRuns = []
    for result in results:
        if 'gold' in result[STATUS].lower() or 'pro' in result[STATUS].lower():
            try:
                silverRuns.append(float(result[RUN_2_KTS]))
            except ValueError:
                silverRuns.append(0)

    if len(silverRuns) > 0:
        silverRuns = sorted(silverRuns, reverse=True)

        percent25 = math.ceil(len(silverRuns) / 4)
        percent50 = math.ceil(len(silverRuns) / 2)

        if silverRuns[percent25 - 1] >= 28 or silverRuns[percent50 - 1] >= 26:
            newRule = True
        else:
            newRule = False

        link = '[{}](https://logiqx.github.io/wsw-results/results/{}/{}/ukwa.html)'.format(session, session[:4], session)
        
        if oldRule == True and newRule == False:
            print('-', link, 'new rules deficient', len(results), silverRuns[percent25 - 1], silverRuns[percent50 - 1])

        if oldRule == False and newRule == True:
            print('-', link, 'new rules help', len(results), silverRuns[percent25 - 1], silverRuns[percent50 - 1])
            pass

In [5]:
def processEvent(year):
    '''processYear processes all of the sessions for an event'''

    csvFiles = sorted(glob.glob(os.path.join(PROJDIR, 'events', str(year), 'sailwave', '*', '*Men*.csv')))
    
    for csvFile in csvFiles:
        if "Wing" not in csvFile:
            processSession(year, csvFile)

In [6]:
def main():
    for year in range(2000, 2050):
        processEvent(year)
      
    print('All done')

if __name__ == '__main__':
    main()

- [20111019](https://logiqx.github.io/wsw-results/results/2011/20111019/ukwa.html) new rules help 18 27.63 26.12
- [20121009](https://logiqx.github.io/wsw-results/results/2012/20121009/ukwa.html) new rules help 19 28.99 25.57
- [20121009](https://logiqx.github.io/wsw-results/results/2012/20121009/ukwa.html) new rules help 19 29.1 26.38
- [20121010](https://logiqx.github.io/wsw-results/results/2012/20121010/ukwa.html) new rules help 13 29.56 27.57
- [20141018](https://logiqx.github.io/wsw-results/results/2014/20141018/ukwa.html) new rules help 19 28.78 26.73
- [20141018](https://logiqx.github.io/wsw-results/results/2014/20141018/ukwa.html) new rules help 15 28.54 27.31
- [20151004](https://logiqx.github.io/wsw-results/results/2015/20151004/ukwa.html) new rules help 21 28.39 25.57
- [20151005](https://logiqx.github.io/wsw-results/results/2015/20151005/ukwa.html) new rules help 24 28.3 26.72
- [20221018](https://logiqx.github.io/wsw-results/results/2022/20221018/ukwa.html) new rules help 

## All Done!